In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016AB36DB730>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016AB372C730>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Method 1 - When the content is not so huge

In [20]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

### SystemMessage = Instructions given to the model
### HumanMessage = User input
### AIMessage = Model output

In [21]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.

In our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds of the government offices for this purpose. The government should reach him. Since you have assigned this responsibility to me, about four crore families have got their ‘pucca’ houses. However, I have encountered cases where someone is left out of the government benefits. Therefore, I have decided to tour the country again, to listen to people’s experiences with government schemes, to understand whether they received the intended benefits, and to ensure that the programs are reaching everyone as planned without paying any bribes. We will get the real picture if we visit them again. Therefore, this ‘Viksit Bharat Sankalp Yatra’ is, in a way, my own examination. I want to hear from you and the people across the country whether what I envisioned and the work I have been doing aligns with reality and whether it has reached those for whom it was meant.

It is crucial to check whether the work that was supposed to happen has indeed taken place. I recently met some individuals who utilized the Ayushman card to get treatment for serious illnesses. One person met with a severe accident, and after using the card, he could afford the necessary operation, and now he is recovering well. When I asked him, he said: “How could I afford this treatment? Now that there is the Ayushman card, I mustered courage and underwent an operation. Now I am perfectly fine.”  Such stories are blessings to me.

The bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel satisfied that 50 or 100 people who were supposed to get the funds have got it. The funds meant for a thousand villages have been released. But their job satisfaction peaks when they hear that their work has directly impacted someone’s life positively. When they see the tangible results of their efforts, their enthusiasm multiplies. They feel satisfied. Therefore, ‘Viksit Bharat Sankalp Yatra’ has had a positive impact on government officers. It has made them more enthusiastic about their work, especially when they witness the tangible benefits reaching the people. Officers now feel satisfied with their work, saying, “I made a good plan, I created a file, and the intended beneficiaries received the benefits.” When they find that the money has reached a poor widow under the Jeevan Jyoti scheme and it was a great help to her during her crisis, they realise that they have done a good job. When a government officer listens to such stories, he feels very satisfied.

There are very few who understand the power and impact of the ‘Viksit Bharat Sankalp Yatra’. When I hear people connected to bureaucratic circles talking about it, expressing their satisfaction, it resonates with me. I’ve heard stories where someone suddenly received 2 lakh rupees after the death of her husband, and a sister mentioned how the arrival of gas in her home transformed her lives. The most significant aspect is when someone says that the line between rich and poor has vanished. While the slogan ‘Garibi Hatao’ (Remove Poverty) is one thing, but the real change happens when a person says, “As soon as the gas stove came to my house, the distinction between poverty and affluence disappeared.
"""

In [22]:
speech

'\nPeople across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.\n\nIn our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds of the gove

In [23]:
chat_message = [
    SystemMessage(content="You are expert with experise in summarizing speeched"),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech:\n Text: {speech}"),
]

In [24]:
llm.get_num_tokens(speech)

909

In [25]:
llm(chat_message)

AIMessage(content='This speech highlights the "Viksit Bharat Sankalp Yatra" (Developed India Resolution Journey) as a platform for evaluating the effectiveness of government schemes. \n\nThe speaker, a Member of Parliament, emphasizes the importance of ensuring that government benefits reach the intended beneficiaries without hassle. They share personal anecdotes and experiences of people positively impacted by government programs like the Prime Minister\'s housing scheme and Ayushman Bharat. \n\nThe speaker believes that witnessing the tangible results of their work motivates government officials and reinforces the impact of these initiatives. They conclude by emphasizing the significance of these stories in bridging the gap between the rich and poor, highlighting the transformative power of government programs.  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 889, 'total_tokens': 1027, 'completion_time': 0.250909091, 'prompt_

In [26]:
llm(chat_message).content

'This speech emphasizes the importance of delivering government benefits directly to the intended beneficiaries.  The speaker, a Member of Parliament, highlights his personal commitment to ensuring that schemes like the Pradhan Mantri Awas Yojana (housing scheme) and the Ayushman card (health insurance) reach the people who need them most. \n\nHe describes his "Viksit Bharat Sankalp Yatra" (Developed India Resolution Journey) as a way to personally assess the effectiveness of these programs and gather firsthand accounts from citizens. He shares inspiring stories of individuals whose lives have been positively impacted by government schemes, emphasizing the tangible benefits they have received.\n\nThe speaker also underscores the positive impact of the yatra on government officials, who find satisfaction in witnessing the direct impact of their work on people\'s lives. Ultimately, the speech aims to demonstrate the real-world difference government initiatives can make when they reach th

### Method 2 - When the content is so huge

In [27]:
### Prompt Template

from langchain.chains import LLMChain
from langchain import PromptTemplate

generic_template = """

    Write a summary of the following speech:
    Speech: {speech}
    Translate the precise summary to {language}

"""

prompt = PromptTemplate(
    input_variables=["speech", "language"],
    template=generic_template
)

prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\n\n    Write a summary of the following speech:\n    Speech: {speech}\n    Translate the precise summary to {language}\n\n')

In [28]:
complete_prompt = prompt.format(speech=speech, language="Hindi")
complete_prompt

'\n\n    Write a summary of the following speech:\n    Speech: \nPeople across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.\n\nIn our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get t

In [29]:
llm.get_num_tokens(complete_prompt)

939

In [31]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)
summary = llm_chain.run({'speech': speech, 'language': "Hindi"})
summary

C:\Users\adity\AppData\Local\Temp\ipykernel_17104\1282165269.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\adity\AppData\Local\Temp\ipykernel_17104\1282165269.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = llm_chain.run({'speech': speech, 'language': "Hindi"})


"यह भाषण एक सामाजिक कार्यक्रम, 'विकसित भारत संकल्प यात्रा' की सफलता में जनभागीदारी के बारे में है। भाषणकर्ता, एक सांसद, इस कार्यक्रम में अपनी भागीदारी को अपने उत्तरदायित्व के रूप में बताते हैं और यह सुनिश्चित करने के लिए देशभर में यात्रा कर रहे हैं कि सरकार की योजनाएं सही लाभार्थियों तक पहुँचें। \n\nउन्होंने 'प्रधानमंत्री आवास योजना' के उदाहरण देकर बताया कि जहाँ लाखों परिवारों को 'पक्का' घर मिले हैं, वहाँ भी कुछ लोगों को लाभ से वंचित रहने की खबर मिली है। इस यात्रा के माध्यम से, भाषणकर्ता लोगों की सरकार की योजनाओं के प्रति प्रतिक्रिया सुनना चाहते हैं और भ्रष्टाचार से मुक्त लाभ वितरण सुनिश्चित करना चाहते हैं।\n\nभाषणकर्ता 'आयुष्मान कार्ड' के माध्यम से लोगों को स्वास्थ्य सेवा लाभ मिलने की कहानियों से अपना उत्साह व्यक्त करते हैं। उन्होंने कहा कि यह यात्रा न सिर्फ़ लाभार्थियों के लिए बल्कि सरकारी अधिकारियों के लिए भी प्रेरणादायी है क्योंकि वे अपने काम के सीधे परिणाम देखते हैं। \n\nभाषणकर्ता कहानियों के माध्यम से इस यात्रा के सकारात्मक प्रभाव को दर्शाते हुए कहते हैं कि यह यात्रा अमीर और गरीब 